In [1]:
import pandas as pd
pd.set_option('display.max_columns', 250)

import numpy as np

from scipy.sparse import coo_matrix, csr_matrix, hstack

import seaborn as sns
import matplotlib.pyplot as plt

#imputers
from impyute.imputation.cs import mice, fast_knn 
from sklearn.impute import SimpleImputer 

#encoders
import category_encoders as ce

from sklearn.base import TransformerMixin
from sklearn.model_selection import train_test_split, StratifiedKFold, KFold
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn import metrics
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier

from catboost import CatBoostClassifier


import itertools
import pickle



In [2]:
SEED = 42

In [3]:
def train_model_cv_cl(X_train, y_train, X_test, params, 
                      encoder = None, 
                      model_type = 'sklearn', eval_metric = 'auc', 
                      sk_model = None, 
                      n_folds = 5,
                      n_estimators = 1000,
                      n_jobs = -1,
                      verbose = 250,
                      early_stopping_rounds = 200
                     ):
    if encoder is not None:
        X_train = encoder.fit_transform(X_train, y_train)
        X_test = encoder.transform(X_test)
    
    # set up scoring parameters for different models
    metrics_dict = {'auc': {'lgb': 'auc',         
                            'cat': 'AUC',
                            'sklearn': metrics.roc_auc_score,
                            'xgb': 'auc'}  
                   }
    cv_scores = []
    result_dict = {}
    pred_test = np.zeros(X_test.shape[0])
    pred_train = np.zeros(X_train.shape[0]) #collect all validation predictions over all folds
    
    fold_method = StratifiedKFold(n_splits = n_folds, shuffle = True, random_state = SEED)
    for fold_n, (dev_index, val_index) in enumerate(fold_method.split(X_train, y_train)):
        try: #if data scr or numpy matrices
            X_dev, X_val = X_train[dev_index], X_train[val_index]
            y_dev, y_val = y_train[dev_index], y_train[val_index]
            
        except: #if x data is pandas dataframes
            X_dev, X_val = X_train.iloc[dev_index], X_train.iloc[val_index]
            y_dev, y_val = y_train[dev_index], y_train[val_index]
              
        if model_type == 'sklearn':
            model = sk_model(**params)
            model.fit(X_dev, y_dev)
            pred_y_val = model.predict_proba(X_val)[:, 1]
            pred_y_test = model.predict_proba(X_test)[:, 1]
            
            score = metrics_dict[eval_metric][model_type](y_val, pred_y_val)
            print(f'Fold {fold_n}. {eval_metric}: {score:.6f}.')
            #print('')
            
        if model_type == 'lgb':
            model = lgb.LGBMClassifier(**params, n_estimators=n_estimators, n_jobs = n_jobs)
            model.fit(X_dev, y_dev, eval_set = [(X_val, y_val)], 
                      eval_metric = metrics_dict[eval_metric][model_type],
                      verbose=verbose, 
                      early_stopping_rounds=early_stopping_rounds)
            pred_y_val = model.predict_proba(X_val)[:, 1]
            pred_y_test = model.predict_proba(X_test, num_iteration=model.best_iteration_)[:, 1]
            
        if model_type == 'xgb':
            dev_data = xgb.DMatrix(data = X_dev, label = y_dev)
            valid_data = xgb.DMatrix(data = X_val, label = y_val)            
            watchlist = [(dev_data, 'dev_data'), (valid_data, 'valid_data')]
            
            param = params #outside parameters
            param['eval_metric'] = metrics_dict[eval_metric][model_type] #add parameter from metric_dict
            
            model = xgb.train(dtrain = dev_data, evals=watchlist,
                              num_boost_round = n_estimators, 
                              early_stopping_rounds=early_stopping_rounds, 
                              verbose_eval=verbose, params=params)            
            pred_y_val = model.predict(xgb.DMatrix(X_val), ntree_limit=model.best_ntree_limit)
            pred_y_test = model.predict(xgb.DMatrix(X_test), ntree_limit=model.best_ntree_limit)
            
        if model_type == 'cat':
            model = CatBoostClassifier(iterations=n_estimators, eval_metric=metrics_dict[eval_metric][model_type], **params,
                                      loss_function='Logloss', od_wait = early_stopping_rounds)
            model.fit(X_dev, y_dev, eval_set=(X_val, y_val), use_best_model=True, verbose=verbose)
            pred_y_val = model.predict_proba(X_val)[:, 1]
            pred_y_test = model.predict_proba(X_test)[:, 1]
            
        
        pred_train[val_index] += pred_y_val
        pred_test += pred_y_test
        cv_scores.append(metrics_dict[eval_metric]['sklearn'](y_val, pred_y_val))
        
    pred_test /= n_folds
    result_dict['predictions'] = pred_test
    result_dict['overall_val_predictions'] = pred_y_val
    result_dict['cv_scores'] = cv_scores
    
    print('CV mean score: {0:.6f}, std: {1:.6f}.'.format(np.mean(cv_scores), np.std(cv_scores)))
    
    return result_dict

## Загрузка данных

In [4]:
churn_data_train = pd.read_csv('./Data/kaggle_data/orange_small_churn_train_data.csv')
churn_data_test = pd.read_csv('./Data/kaggle_data/orange_small_churn_test_data.csv')

In [7]:
churn_data_train['labels'] = churn_data_train['labels'] > 0 #convert (-1, 1) to (True, False)

In [8]:
churn_data_train.head()

,ID,Var1,Var2,Var3,Var4,Var5,Var6,Var7,Var8,Var9,Var10,Var11,Var12,Var13,Var14,Var15,Var16,Var17,Var18,Var19,Var20,Var21,Var22,Var23,Var24,Var25,Var26,Var27,Var28,Var29,Var30,Var31,Var32,Var33,Var34,Var35,Var36,Var37,Var38,Var39,Var40,Var41,Var42,Var43,Var44,Var45,Var46,Var47,Var48,Var49,Var50,Var51,Var52,Var53,Var54,Var55,Var56,Var57,Var58,Var59,Var60,Var61,Var62,Var63,Var64,Var65,Var66,Var67,Var68,Var69,Var70,Var71,Var72,Var73,Var74,Var75,Var76,Var77,Var78,Var79,Var80,Var81,Var82,Var83,Var84,Var85,Var86,Var87,Var88,Var89,Var90,Var91,Var92,Var93,Var94,Var95,Var96,Var97,Var98,Var99,Var100,Var101,Var102,Var103,Var104,Var105,Var106,Var107,Var108,Var109,Var110,Var111,Var112,Var113,Var114,Var115,Var116,Var117,Var118,Var119,Var120,Var121,Var122,Var123,Var124,Var125,Var126,Var127,Var128,Var129,Var130,Var131,Var132,Var133,Var134,Var135,Var136,Var137,Var138,Var139,Var140,Var141,Var142,Var143,Var144,Var145,Var146,Var147,Var148,Var149,Var150,Var151,Var152,Var153,Var154,Var155,Var156,Var157,Var158,Var159,Var160,Var161,Var162,Var163,Var164,Var165,Var166,Var167,Var168,Var169,Var170,Var171,Var172,Var173,Var174,Var175,Var176,Var177,Var178,Var179,Var180,Var181,Var182,Var183,Var184,Var185,Var186,Var187,Var188,Var189,Var190,Var191,Var192,Var193,Var194,Var195,Var196,Var197,Var198,Var199,Var200,Var201,Var202,Var203,Var204,Var205,Var206,Var207,Var208,Var209,Var210,Var211,Var212,Var213,Var214,Var215,Var216,Var217,Var218,Var219,Var220,Var221,Var222,Var223,Var224,Var225,Var226,Var227,Var228,Var229,Var230,labels
0,0,NaN,NaN,NaN,NaN,NaN,3052.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,480.0,600.0,NaN,20.0,480.0,NaN,NaN,200.00,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,82752.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.907926,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,34,NaN,NaN,716008.0,NaN,0.0,NaN,NaN,14599.92,NaN,5.0,NaN,32.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,144.0,NaN,NaN,144.0,-1209960.0,NaN,NaN,NaN,NaN,NaN,1660.0,NaN,NaN,NaN,66.0,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,0.0,326915.0,604276.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,9.0,NaN,NaN,NaN,NaN,389396.0,NaN,NaN,NaN,2313888.0,NaN,NaN,NaN,NaN,NaN,NaN,28.0,NaN,NaN,599532.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NESt0G8EIb,AERks4l,NaN,taul,1K8T,0LaQ,UaKK0yW,I1sFbv_0IT,NaN,NaN,EkHG,9_Y1,k13i,09_Q,IYzP,GjJ35utlTa_GNSvxxpb9ju,kIsH,NaN,uKAI,L84s,JBfYVit4g8,NaN,NaN,NaN,TDctq2l,KmRo,cJvF,FzaX,hLKtJ9p,Al6ZaUT,vr93T2a,LM8l689qOp,NaN,NaN,fKCe,02N6s8f,xwM2aC7IdeMC0,NaN,NaN,False
1,1,NaN,NaN,NaN,NaN,NaN,1813.0,7.0,NaN,NaN,NaN,NaN,NaN,636.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,212.0,265.0,NaN,2.0,128.0,NaN,NaN,166.56,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,2706120.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.870327,NaN,NaN,NaN,NaN,NaN,NaN,NaN,27.0,NaN,NaN,NaN,NaN,NaN,NaN,3.0,128,0.0,NaN,1661128.0,NaN,0.0,NaN,NaN,67529.09,NaN,25.0,NaN,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,32289.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,80.0,NaN,NaN,72.0,417932.0,NaN,NaN,NaN,NaN,NaN,1025.0,NaN,NaN,NaN,66.0,NaN,24912.0,40.0,NaN,NaN,NaN,NaN,NaN,0.0,1934460.0,349568.0,NaN,NaN,NaN,NaN,NaN,205.0,NaN,NaN,0.0,18.0,NaN,NaN,NaN,NaN,735.0,NaN,NaN,NaN,6502680.0,NaN,NaN,NaN,NaN,NaN,NaN,14.0,NaN,NaN,364182.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,276.0,NaN,NaN,P1WvyxLp3Z,2Knk1KF,NaN,taul,1K8T,YFAj,Bnunsla,o64y9zI,DlISMzi,NaN,JDd6,9_Y1,FbIm,VpdQ,haYg,me75fM6ugJ,kIsH,NaN,uKAI,L84s,XfqtO3UdzaXh_,NaN,I_mbgcr,NaN,XTbqizz,qMoY,cJvF,FzaX,hN8KpA1,oslk,6hQ9lNX,LM8l689qOp,NaN,ELof,xb3V,RAYp,55YFVY9,mj86,NaN,False
2,2,NaN,NaN,NaN,NaN,NaN,1953.0,7.0,NaN,NaN,NaN,NaN,NaN,448.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,176.0,220.0,NaN,0.0,72.0,NaN,NaN,311.76,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,4698780.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.981628,NaN,NaN,NaN,NaN,NaN,Na

In [9]:
churn_data_test.head()

,ID,Var1,Var2,Var3,Var4,Var5,Var6,Var7,Var8,Var9,Var10,Var11,Var12,Var13,Var14,Var15,Var16,Var17,Var18,Var19,Var20,Var21,Var22,Var23,Var24,Var25,Var26,Var27,Var28,Var29,Var30,Var31,Var32,Var33,Var34,Var35,Var36,Var37,Var38,Var39,Var40,Var41,Var42,Var43,Var44,Var45,Var46,Var47,Var48,Var49,Var50,Var51,Var52,Var53,Var54,Var55,Var56,Var57,Var58,Var59,Var60,Var61,Var62,Var63,Var64,Var65,Var66,Var67,Var68,Var69,Var70,Var71,Var72,Var73,Var74,Var75,Var76,Var77,Var78,Var79,Var80,Var81,Var82,Var83,Var84,Var85,Var86,Var87,Var88,Var89,Var90,Var91,Var92,Var93,Var94,Var95,Var96,Var97,Var98,Var99,Var100,Var101,Var102,Var103,Var104,Var105,Var106,Var107,Var108,Var109,Var110,Var111,Var112,Var113,Var114,Var115,Var116,Var117,Var118,Var119,Var120,Var121,Var122,Var123,Var124,Var125,Var126,Var127,Var128,Var129,Var130,Var131,Var132,Var133,Var134,Var135,Var136,Var137,Var138,Var139,Var140,Var141,Var142,Var143,Var144,Var145,Var146,Var147,Var148,Var149,Var150,Var151,Var152,Var153,Var154,Var155,Var156,Var157,Var158,Var159,Var160,Var161,Var162,Var163,Var164,Var165,Var166,Var167,Var168,Var169,Var170,Var171,Var172,Var173,Var174,Var175,Var176,Var177,Var178,Var179,Var180,Var181,Var182,Var183,Var184,Var185,Var186,Var187,Var188,Var189,Var190,Var191,Var192,Var193,Var194,Var195,Var196,Var197,Var198,Var199,Var200,Var201,Var202,Var203,Var204,Var205,Var206,Var207,Var208,Var209,Var210,Var211,Var212,Var213,Var214,Var215,Var216,Var217,Var218,Var219,Var220,Var221,Var222,Var223,Var224,Var225,Var226,Var227,Var228,Var229,Var230
0,0,NaN,NaN,NaN,NaN,NaN,1225.0,7.0,NaN,NaN,NaN,NaN,NaN,2352.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,180.0,225.0,NaN,0.0,56.0,NaN,NaN,388.08,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,6726960.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.887844,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,124,105.0,NaN,172160.0,NaN,0.0,NaN,NaN,171040.20,NaN,40.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.0,NaN,NaN,16.0,184566.8,NaN,NaN,NaN,NaN,NaN,635.0,NaN,NaN,NaN,84.0,NaN,1224.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1146085.0,1012944.0,NaN,NaN,NaN,NaN,NaN,6310.0,NaN,NaN,0.0,9.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,10278280.0,NaN,NaN,NaN,NaN,NaN,NaN,28.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,P1WvyxLp3Z,2Knk1KF,NaN,taul,1K8T,0Xwj,PHNvXy8,xUOdRmdN20,IBm9AvG,NaN,skcI,HLqf,vzJD,NaN,lVqb,NKv3VA1BpP,sBgB,NaN,uKAI,L84s,Ie_5MZs,NaN,xRz9wph,NaN,CWNY2py,jgOV,UYBR,FzaX,UF16siJ,zCkv,APgdzOv,jySVZNlOJy,NaN,ELof,xb3V,6fzt,Zy3gnGM,NaN,NaN
1,1,NaN,NaN,NaN,NaN,NaN,896.0,14.0,NaN,NaN,NaN,NaN,NaN,1560.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,112.0,140.0,NaN,0.0,96.0,NaN,NaN,133.12,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.320230,NaN,NaN,NaN,NaN,NaN,NaN,NaN,36.0,NaN,NaN,NaN,NaN,NaN,NaN,6.0,44,217.0,NaN,0.0,NaN,0.0,NaN,NaN,12848.37,NaN,0.0,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,166446.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,0.0,82004.4,NaN,NaN,NaN,NaN,NaN,340.0,NaN,NaN,NaN,0.0,NaN,29691.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,606020.0,121946.0,NaN,NaN,NaN,NaN,NaN,3675.0,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,777520.0,NaN,NaN,NaN,NaN,NaN,NaN,28.0,NaN,NaN,73218.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,bZkvyxLkBI,RO12,NaN,taul,1K8T,0Xwj,6KF0k8W,V4E_TU9097,NaN,NaN,CZsd,F3hy,vzJD,09_Q,IYzP,me75fM6ugJ,kIsH,NaN,uKAI,L84s,NhsEn4L,NaN,NaN,NaN,TDcECyH,PDRj,cJvF,FzaX,ot6oLzk,oslk,IIvC99a,LM8l689qOp,NaN,NaN,xb3V,RAYp,F2FyR07IdsN7I,NaN,NaN
2,2,NaN,NaN,NaN,NaN,NaN,791.0,7.0,NaN,NaN,NaN,NaN,NaN,2824.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,172.0,215.0,NaN,4.0,16.0,NaN,NaN,324.48,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,2988486.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,152312.0,NaN,NaN,NaN,NaN,NaN,0.628285,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.0,NaN,NaN,NaN,NaN,NaN,NaN,3.0,88,329.0,NaN,923712.0,NaN,0.0

In [ ]:
print(churn_data_train.shape, churn_data_test.shape)

In [ ]:
churn_data_train.drop(columns = ['ID'], axis = 0, inplace = True)
churn_data_test.drop(columns = ['ID'], axis = 0, inplace = True)

Первые 190 признаков - числовые, остальные 40 - категориальные.

## Подготовка данных

### Отбор признаков по колчеству пропущенных значений

Определим, есть ли в train полностью пустые столбцы. 

In [ ]:
const_feature = churn_data_train.isna().sum(axis = 0) == len(churn_data_train)
useless_features = const_feature[const_feature == True].index
print('Полностью пустые признаки: \n', useless_features.to_list())

In [ ]:
num_features_names = ['Var{}'.format(i) for i in range(1, 191) if 'Var{}'.format(i) not in useless_features]
cat_features_names = ['Var{}'.format(i) for i in range(191, 231) if 'Var{}'.format(i) not in useless_features]

print('Количество числовых признаков:', len(num_features_names))
print('Количество категориальных признаков:', len(cat_features_names))

In [ ]:
num_data = churn_data.loc[:, num_features_names]
cat_data = churn_data.loc[:, cat_features_names]

**Подберем порог отбрасывания числовых признаков по количеству пустых объектов**

In [ ]:
feature_amount = []
for threshold in range(0, 40000, 1000):
    nan_counter = num_data.isna().sum(axis = 0)
    f_amount = len(nan_counter[nan_counter < threshold])
    feature_amount.append(f_amount)

plt.figure(figsize = (10, 5))
plt.plot(range(0, 40000, 1000), feature_amount)
plt.title('Изменение количества числовых признаков в зависимости от фильтрующего порога')
plt.xlabel('Пороговое количество nan объектов')
plt.ylabel('Количество отобранных признаков')
plt.show()

In [ ]:
NUM_THRESHOLD = 25000

In [ ]:
nan_count = num_data.isna().sum(axis = 0).to_frame(name = 'count')
useful_num_featues = nan_count[nan_count['count'] < NUM_THRESHOLD].index

num_data = num_data[useful_num_featues]

In [ ]:
print('Итоговое количество отобранных числовых признаков:', num_data.shape[1])

**Аналогично подберем порог для категориальных признаков**

In [ ]:
feature_amount = []
for threshold in range(0, 40000, 1000):
    nan_counter = cat_data.isna().sum(axis = 0)
    f_amount = len(nan_counter[nan_counter < threshold])
    feature_amount.append(f_amount)

plt.figure(figsize = (10, 5))
plt.plot(range(0, 40000, 1000), feature_amount)
plt.title('Изменение количества категориальных признаков в зависимости от фильтрующего порога')
plt.xlabel('Пороговое количество not nan объектов')
plt.ylabel('Количество отобранных признаков')
plt.show()

Обработаем все категориальные признаки

### Разбиение данных

In [ ]:
full_data = pd.concat([num_data, cat_data, churn_data['label']], axis = 1)
full_data.head()

In [ ]:
num_col_names = num_data.columns
cat_col_names = cat_data.columns

print('Число сырых числовых признаков', len(num_col_names))
print('Число сырых категориальных признаков', len(cat_col_names))

In [ ]:
X = full_data.drop(columns = ['label'])
y = full_data['label']

X_train_raw, X_test_raw, y_train, y_test = train_test_split(X, y, stratify = y, test_size = 0.2, random_state = SEED)

y_train = y_train.values
y_test = y_test.values

### Data imputation

**Simple mean imputer**

In [ ]:
imp = SimpleImputer(missing_values = np.nan, strategy = 'mean')
X_train_num_mean = imp.fit_transform(X_train_raw[num_col_names])
X_test_num_mean = imp.transform(X_test_raw[num_col_names])

X_train_num_mean = pd.DataFrame(X_train_num_mean, columns = num_col_names)
X_test_num_mean = pd.DataFrame(X_test_num_mean, columns = num_col_names)

In [ ]:
print(X_train_num_mean.shape)
print(X_test_num_mean.shape)

In [ ]:
print(X_train_num_mean.isnull().sum().sum())
print(X_test_num_mean.isnull().sum().sum())

**Fillna + additional feature**

In [ ]:
def fillna_imputer(df):
    column_names = df.columns.to_list()
    result = pd.DataFrame()
    for name in column_names:
        result[name] = df[name].fillna(value = 'Nan')
        result[name + '_bool'] = df[name].isna()
    return result.reset_index(drop = True)

In [ ]:
X_train_cat_fillna = fillna_imputer(X_train_raw[cat_col_names])
X_test_cat_fillna = fillna_imputer(X_test_raw[cat_col_names])

In [ ]:
print(X_train_cat_fillna.shape)
print(X_test_cat_fillna.shape)

In [ ]:
print(X_train_cat_fillna.isnull().sum().sum())
print(X_test_cat_fillna.isnull().sum().sum())

**Формирование датасетов**

In [ ]:
X_train = pd.concat([X_train_num_mean, X_train_cat_fillna], axis = 1)
X_test =  pd.concat([X_test_num_mean, X_test_cat_fillna], axis = 1)

In [ ]:
print('Типы данных в получившихся датасетах:', list(X_train.dtypes.unique()))
print('Размеры датасетов:', (X_train.shape, X_test.shape))

### Label encoding

In [ ]:
catboost_encoder = ce.CatBoostEncoder(random_state = SEED)
X_train_cat = catboost_encoder.fit_transform(X_train, y_train)
X_test_cat = catboost_encoder.transform(X_test)

In [ ]:
# set back old dataframes columns names
X_train_cat.columns = X_train.columns
X_test_cat.columns = X_test.columns

In [ ]:
X_train_cat

### Naive model predictions

**Standard Scaling**

In [ ]:
scaler = StandardScaler()
X_train_sc = scaler.fit_transform(X_train_cat)
X_test_sc = scaler.fit_transform(X_test_cat)

#### Выбор оптимального набора даных для различных моделей

##### Линейные модели

Линейные модели есть смысл запускать как на  ohe, так и на catboost комплектах данных

###### LogisticRegression

In [ ]:
logistic_model_dict = {}

In [ ]:
for key in data_encoders_dict['ohe'].keys():
    print(key)
    X_train, X_test, column_names = data_encoders_dict['ohe'][key]
    linear_model_dict['ohe_' + key] = train_model_cv_cl(X_train, y_train, X_test, 
                                                        model_type = 'sklearn', eval_metric = 'auc', 
                                                        sk_model = LogisticRegression,
                                                        params = {'solver': 'lbfgs', 
                                                                  'max_iter': 1000, 
                                                                  'n_jobs': -1}) 
    print('------------------------------------------')

In [ ]:
for key in data_encoders_dict['catboost'].keys():
    print(key)
    X_train, X_test, column_names = data_encoders_dict['catboost'][key]
    linear_model_dict['catboost_' + key] = train_model_cv_cl(X_train, y_train, X_test, 
                                                        model_type = 'sklearn', eval_metric = 'auc', 
                                                        sk_model = LogisticRegression,
                                                        params = {'solver': 'lbfgs', 
                                                                  'max_iter': 1000, 
                                                                  'n_jobs': -1}) 
    print('------------------------------------------')

Для логистической регресии оптимальной оказалась стратегия CatBoost encoding + mean_fillna imputing

###### SVM (SVC)

SVM (SVC)

In [ ]:
LinearSVC_model_dict = {}

In [ ]:
#Linear SVC doesn`t support 'predict_proba'

# for key in data_encoders_dict['ohe'].keys():
#     print(key)
#     X_train, X_test, column_names = data_encoders_dict['ohe'][key]
#     SVM_model_dict['ohe_' + key] = train_model_cv_cl(X_train, y_train, X_test, 
#                                                         model_type = 'sklearn', eval_metric = 'auc', 
#                                                         sk_model = LinearSVC,
#                                                         params = {'random_state': SEED,                          
#                                                                   'class_weight': 'balanced'}) 
#     print('------------------------------------------')

##### Нелинейные модели

###### Random Forest Classifier

In [ ]:
random_forest_dict = {}

In [ ]:
for key in data_encoders_dict['catboost'].keys():
    print(key)
    X_train, X_test, column_names = data_encoders_dict['catboost'][key]
    random_forest_dict['catboost_' + key] = train_model_cv_cl(X_train, y_train, X_test, 
                                                        model_type = 'sklearn', eval_metric = 'auc', 
                                                        sk_model = RandomForestClassifier,
                                                        params = {'n_estimators': 250, 
                                                                  'class_weight': 'balanced', 
                                                                  'n_jobs': -1}) 
    print('------------------------------------------')

###### CatBoost Classifier

In [ ]:
catboost_dict = {}

In [ ]:
for key in data_encoders_dict['catboost'].keys():
    print(key)
    X_train, X_test, column_names = data_encoders_dict['catboost'][key]
    catboost_dict['catboost_' + key] = train_model_cv_cl(X_train, y_train.astype('int'), X_test, 
                                                        model_type = 'cat', eval_metric = 'auc', 
                                                        params = {}) 
    print('------------------------------------------')

Лучше всего показал себя градиентный бустинг с применением mean_fillna кодирования.